In [1]:
import calendar  # .monthrange(year, month) : 해당년도, 해당월의 날짜수
from bs4 import BeautifulSoup  # html 분석
from selenium import webdriver as wd  # 크롤링 웹 접속


import xml.etree.ElementTree as elemTree

import urllib
from urllib.error import HTTPError,URLError
from urllib.request import urlretrieve
import urllib.parse
from urllib.request import urlopen

import requests

import time
from tqdm import tqdm

from selenium.webdriver.common.by import By

import pandas as pd



In [2]:
gu = ["종로구", "중구", "용산구", "성동구", "중랑구", "강북구", "도봉구", 
      "노원구", "은평구", "양천구", "구로구", "서초구", "강남구", "송파구", "강동구"]

b_dong = ["연지동", "효제동", "종로5가", "종로6가", "충신동", "무교동","다동", "태평로1가", "을지로1가", "을지로2가", 
          "남대문로1가", "삼각동", "수하동", "장교동", "수표동", "남대문로2가", "회현동1가", "회현동2가", "회현동3가", 
          "충무로1가", "충무로2가", "명동1가", "명동2가", "남산동1가", "남산동2가", "남산동3가", "저동1가", "예장동", 
          "을지로4가" ,"을지로5가", "주교동", "방산동", "을지로3가", "입정동", "산림동", "초동", "인현동1가", "저동2가", 
          "이촌동", "금호동1가", "면목동", "미아동", "창동", "월계동", "하계동", "중계동", "상계동", "진관동", "신월동", 
          "등촌동", "가양동", "염창동", "방화동", "구로동", "서초동", "반포동", "개포동", "신정동", "대치동", "일원동", 
          "가락동", "문정동", "장지동", "잠실동", "상일동", "둔촌동"]

In [3]:
# 단독다가구 매매
singleHouse_sales = {"종로구" : ["연지동", "효제동", "종로5가", "종로6가", "충신동"],
                     "성동구": ["금호1가동"], "중랑구" : ["면목동"], "강북구" : ["미아동"],
                     "노원구" : ["하계동", "중계동", "상계동"], 
                     "은평구" : ["진관동"], "양천구" : ["신월동", "신정동"],
                     "강서구" : ["가양동", "등촌동", "염창동", "방화동"], 
                     "구로구" : ["구로동"], "서초구" : ["서초동"], "송파구" : ["가락동"], "강동구" : ["강일동"]}


# 단독다가구 전월세
singleHouse_rent = {"종로구" : ["연지동", "효제동", "종로5가", "종로6가", "충신동"], 
                    "중랑구" : ["면목동"], "노원구" : ["중계동"], "강서구" : ["가양동"], 
                    "구로구" : ["구로동"], "서초구" : ["반포동"] }

# 1. 단독 다가구 매매

In [219]:
def house_sales_info(gu, dong, result):
    df = pd.DataFrame(columns=["구", "법정동", "도로명", "면적", "가격"])
    for i in range(len(result)):
        result_tmp = result[i].text.split(" ")

        if result_tmp[0] != "단독":
            upper_name = result_tmp[0]

        else:
            result_tmp.insert(0, upper_name)

        house_area = result_tmp[3]
        info_house = " ".join(result_tmp[7:-2])

        for info in info_house.split("  "):
            if info.strip() != "":
                house_price = info.split(" ")[1]
                print("구 : ", gu, " / 법정동 : ", dong, " / 도로명 : ", upper_name, 
                      " / 면적 : ", house_area, " / 가격 : ", house_price)
                
                df= df.append({"구" : gu, "법정동" : dong, "도로명" : upper_name,
                                "면적" : house_area, "가격" : house_price}, ignore_index=True)
    return df

In [220]:
driver = wd.Chrome(executable_path="chrome/chromedriver.exe")
driver.implicitly_wait(3)

apt_url = "http://land.seoul.go.kr/land/rtms/rtmsSingleHouse.do" 
driver.get(apt_url)
page = urlopen(apt_url)

In [221]:
# 데이터 프레임 준비
df_houseSales_2019 = pd.DataFrame(columns = ["구", "법정동", "도로명", "면적", "가격"])

In [222]:
# 단독다가구 매매
for gu in tqdm(singleHouse_sales.keys()):
    driver.find_element_by_id("selectJRGubun").send_keys("지번")
    time.sleep(2)

    # 지역선택 - 구 선택
    driver.find_element_by_id("selectSigungu").send_keys(gu)
    time.sleep(2)
    dong_list = singleHouse_sales[gu]
    
    for dong in dong_list:
        # 지역선택 - 동 선택
        driver.find_element_by_id("selectBjdong").send_keys(dong)
        time.sleep(2)

        # 기간선택 - 2019년
        driver.find_element_by_id("selectGubun").send_keys("분기")
        time.sleep(2)
        driver.find_element_by_id("selectYear").send_keys("2019")
        time.sleep(2)
        
        for i in range(1, 5):
            driver.find_element_by_id("selectBoongi").send_keys(str(i)+"분기")
            time.sleep(2)
            # 조회버튼 - click
            xp = """//*[@id="search"]"""
            driver.find_element_by_xpath(xp).click()
            time.sleep(2) 
            
            # 데이터 - table 전부 가져오기
            result_raw = driver.find_element_by_id("resultList")
               
            # 데이터 - row별로 가져오기
            result = result_raw.find_elements_by_tag_name("tr")
            
            if result[0].text == '조회된 내용이 없습니다.':
                continue
            
            # 데이터 - 가공한 후 데이터프레임에 넣기
            new_df = house_sales_info(gu, dong, result)
            
            # 수집된 데이터를 df_aptSales_2019 뒤로 계속 붙여넣기
            df_houseSales_2019 = pd.concat([df_houseSales_2019, new_df], ignore_index=True)


  0%|                                                                                           | 0/12 [00:00<?, ?it/s]

구 :  종로구  / 법정동 :  연지동  / 도로명 :  창경궁로16나길  / 면적 :  24.79  / 가격 :  22,460
구 :  종로구  / 법정동 :  효제동  / 도로명 :  대학로2길  / 면적 :  24.76  / 가격 :  42,000
구 :  종로구  / 법정동 :  효제동  / 도로명 :  대학로2길  / 면적 :  120.35  / 가격 :  125,000
구 :  종로구  / 법정동 :  효제동  / 도로명 :  율곡로14길  / 면적 :  305  / 가격 :  139,531
구 :  종로구  / 법정동 :  효제동  / 도로명 :  김상옥로  / 면적 :  49.59  / 가격 :  178,500
구 :  종로구  / 법정동 :  종로6가  / 도로명 :  낙산성곽서길  / 면적 :  52.89  / 가격 :  49,000
구 :  종로구  / 법정동 :  종로6가  / 도로명 :  종로39길  / 면적 :  212.38  / 가격 :  122,000
구 :  종로구  / 법정동 :  종로6가  / 도로명 :  충신1길  / 면적 :  23.27  / 가격 :  16,000
구 :  종로구  / 법정동 :  충신동  / 도로명 :  충신4길  / 면적 :  103.5  / 가격 :  17,300
구 :  종로구  / 법정동 :  충신동  / 도로명 :  율곡로29길  / 면적 :  26.45  / 가격 :  24,780
구 :  종로구  / 법정동 :  충신동  / 도로명 :  율곡로29길  / 면적 :  35.34  / 가격 :  14,000
구 :  종로구  / 법정동 :  충신동  / 도로명 :  충신길  / 면적 :  114.75  / 가격 :  41,000
구 :  종로구  / 법정동 :  충신동  / 도로명 :  율곡로23가길  / 면적 :  165.93  / 가격 :  53,000
구 :  종로구  / 법정동 :  충신동  / 도로명 :  충신2길  / 면적 :  22.3  / 가격 :  9,600
구 :  종로구  

  8%|██████▊                                                                           | 1/12 [01:57<21:27, 117.08s/it]

구 :  종로구  / 법정동 :  충신동  / 도로명 :  충신4가길  / 면적 :  28.35  / 가격 :  11,000
구 :  종로구  / 법정동 :  충신동  / 도로명 :  충신4나길  / 면적 :  28.8  / 가격 :  12,700
구 :  종로구  / 법정동 :  충신동  / 도로명 :  충신길  / 면적 :  49.44  / 가격 :  8,000
구 :  종로구  / 법정동 :  충신동  / 도로명 :  율곡로23가길  / 면적 :  153.87  / 가격 :  50,500
구 :  종로구  / 법정동 :  충신동  / 도로명 :  율곡로22가길  / 면적 :  278.86  / 가격 :  90,000
구 :  종로구  / 법정동 :  충신동  / 도로명 :  율곡로19가길  / 면적 :  332.28  / 가격 :  89,000


 17%|█████████████▊                                                                     | 2/12 [02:23<14:59, 89.93s/it]

...

 83%|████████████████████████████████████████████████████████████████████▎             | 10/12 [08:34<01:32, 46.05s/it]

구 :  서초구  / 법정동 :  서초동  / 도로명 :  명달로28길  / 면적 :  307.3  / 가격 :  390,000
구 :  서초구  / 법정동 :  서초동  / 도로명 :  사임당로19길  / 면적 :  313.41  / 가격 :  242,000
구 :  서초구  / 법정동 :  서초동  / 도로명 :  서리풀2길  / 면적 :  322.7  / 가격 :  630,000
구 :  서초구  / 법정동 :  서초동  / 도로명 :  사임당로19길  / 면적 :  326.52  / 가격 :  280,000
구 :  서초구  / 법정동 :  서초동  / 도로명 :  남부순환로323길  / 면적 :  354.49  / 가격 :  335,000
구 :  서초구  / 법정동 :  서초동  / 도로명 :  남부순환로315길  / 면적 :  394.23  / 가격 :  370,000
구 :  서초구  / 법정동 :  서초동  / 도로명 :  서리풀3길  / 면적 :  576.55  / 가격 :  314,000
구 :  서초구  / 법정동 :  서초동  / 도로명 :  강남대로37길  / 면적 :  609.88  / 가격 :  439,000
구 :  송파구  / 법정동 :  가락동  / 도로명 :  오금로38가길  / 면적 :  177.21  / 가격 :  124,000
구 :  송파구  / 법정동 :  가락동  / 도로명 :  송이로17길  / 면적 :  247.32  / 가격 :  127,000
구 :  송파구  / 법정동 :  가락동  / 도로명 :  동남로16길  / 면적 :  262.77  / 가격 :  195,000
구 :  송파구  / 법정동 :  가락동  / 도로명 :  중대로9길  / 면적 :  526.4  / 가격 :  200,000
구 :  송파구  / 법정동 :  가락동  / 도로명 :  중대로22길  / 면적 :  188.49  / 가격 :  122,831
구 :  송파구  / 법정동 :  가락동  / 도로명 :  오금로38가길  / 면적 

 92%|███████████████████████████████████████████████████████████████████████████▏      | 11/12 [09:00<00:40, 40.32s/it]

구 :  송파구  / 법정동 :  가락동  / 도로명 :  동남로14길  / 면적 :  84.42  / 가격 :  249,796
구 :  송파구  / 법정동 :  가락동  / 도로명 :  동남로14길  / 면적 :  84.42  / 가격 :  252,127
구 :  송파구  / 법정동 :  가락동  / 도로명 :  송이로19길  / 면적 :  324.07  / 가격 :  210,000
구 :  송파구  / 법정동 :  가락동  / 도로명 :  오금로44길  / 면적 :  345.6  / 가격 :  160,000
구 :  송파구  / 법정동 :  가락동  / 도로명 :  송이로26길  / 면적 :  401.37  / 가격 :  204,000
구 :  송파구  / 법정동 :  가락동  / 도로명 :  송이로20길  / 면적 :  435.43  / 가격 :  180,000
구 :  송파구  / 법정동 :  가락동  / 도로명 :  중대로10길  / 면적 :  485.72  / 가격 :  206,000
구 :  송파구  / 법정동 :  가락동  / 도로명 :  중대로10길  / 면적 :  564.12  / 가격 :  227,000
구 :  송파구  / 법정동 :  가락동  / 도로명 :  송이로23길  / 면적 :  678.77  / 가격 :  230,000
구 :  송파구  / 법정동 :  가락동  / 도로명 :  송이로17길  / 면적 :  1100.7  / 가격 :  360,000


100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [09:27<00:00, 47.29s/it]


In [230]:
driver.close()

In [225]:
df_houseSales_2019.shape

(1498, 5)

In [226]:
(df_houseSales_2019["면적"].str.contains(".")).sum()

1498

In [227]:
(df_houseSales_2019["가격"].str.contains(",")).sum()

1498

In [233]:
df_houseSales_2019.to_csv("data/분석용_데이터/단독_다가구_매매_결측치(cp949).csv", encoding = "cp949", index = False)

# 2. 단독/다가구 전세

In [14]:
def house_rent_info(gu, dong, result):
    df = pd.DataFrame(columns=["구", "법정동", "도로명", "면적", "전세"])
    
    for i in range(len(result)):
        result_tmp = result[i].text.split()

        if result_tmp[0] != "단독다가구":
            upper_name = result_tmp[0]

        else:
            result_tmp.insert(0, upper_name)

        for tmp in result_tmp[3:]:
            if "," in tmp:
                print("구 : ", gu, " / 법정동 : ", dong, " / 도로명 : ", upper_name, 
                        " / 면적 : ", house_area, " / 전세 : ", tmp)
            
                df= df.append({"구" : gu, "법정동" : dong, "도로명" : upper_name,
                                "면적" : house_area, "전세" : tmp}, ignore_index=True)

    return df

In [15]:
# 데이터 프레임 준비
df_houseRent_2019 = pd.DataFrame(columns = ["구", "법정동", "도로명", "면적", "전세"])

In [4]:
driver = wd.Chrome(executable_path="chrome/chromedriver.exe")
driver.implicitly_wait(3)

apt_url = "http://land.seoul.go.kr/land/rtms/rtmsSingleHouse.do" 
driver.get(apt_url)
page = urlopen(apt_url)

xp = """//*[@id="content"]/div[3]/div[1]/section/ul/li[2]/a"""
driver.find_element_by_xpath(xp).click()
time.sleep(2)

In [16]:
for gu in tqdm(singleHouse_rent.keys()):
    driver.find_element_by_id("selectJRGubun").send_keys("지번")
    time.sleep(2)

    # 지역선택 - 구 선택
    driver.find_element_by_id("selectSigungu").send_keys(gu)
    time.sleep(2)
    dong_list = singleHouse_rent[gu]
    
    for dong in dong_list:
        # 지역선택 - 동 선택
        driver.find_element_by_id("selectBjdong").send_keys(dong)
        time.sleep(2)

        # 기간선택 - 2019년
        driver.find_element_by_id("selectGubun").send_keys("분기")
        time.sleep(2)
        driver.find_element_by_id("selectYear").send_keys("2019")
        time.sleep(2)
        
        for i in range(1, 5):
            driver.find_element_by_id("selectBoongi").send_keys(str(i)+"분기")
            time.sleep(2)
            # 조회버튼 - click
            xp = """//*[@id="search"]"""
            driver.find_element_by_xpath(xp).click()
            time.sleep(2)
            
            # 데이터 - table 전부 가져오기
            result_raw = driver.find_element_by_id("resultList")

            # 데이터 - row별로 가져오기
            result = result_raw.find_elements_by_tag_name("tr")
            
            if result[0].text == '조회된 내용이 없습니다.':
                continue
            
            # 데이터 - 가공한 후 데이터프레임에 넣기
            new_df = house_rent_info(gu, dong, result)
            
            # 수집된 데이터를 df_aptSales_2019 뒤로 계속 붙여넣기
            df_houseRent_2019 = pd.concat([df_houseRent_2019, new_df], ignore_index=True)

  0%|                                                                                            | 0/6 [00:00<?, ?it/s]

구 :  종로구  / 법정동 :  효제동  / 도로명 :  종로35길  / 면적 :  99.72  / 전세 :  18,000
구 :  종로구  / 법정동 :  효제동  / 도로명 :  대학로2길  / 면적 :  99.72  / 전세 :  15,000
구 :  종로구  / 법정동 :  효제동  / 도로명 :  대학로2길  / 면적 :  99.72  / 전세 :  12,000
구 :  종로구  / 법정동 :  효제동  / 도로명 :  대학로2길  / 면적 :  99.72  / 전세 :  12,000
구 :  종로구  / 법정동 :  종로6가  / 도로명 :  충신길  / 면적 :  99.72  / 전세 :  13,000
구 :  종로구  / 법정동 :  종로6가  / 도로명 :  낙산성곽서길  / 면적 :  99.72  / 전세 :  12,000
구 :  종로구  / 법정동 :  종로6가  / 도로명 :  율곡로  / 면적 :  99.72  / 전세 :  16,000
구 :  종로구  / 법정동 :  종로6가  / 도로명 :  율곡로29길  / 면적 :  99.72  / 전세 :  10,000
구 :  종로구  / 법정동 :  종로6가  / 도로명 :  충신1길  / 면적 :  99.72  / 전세 :  10,000
구 :  종로구  / 법정동 :  충신동  / 도로명 :  율곡로19길  / 면적 :  99.72  / 전세 :  10,000
구 :  종로구  / 법정동 :  충신동  / 도로명 :  율곡로19길  / 면적 :  99.72  / 전세 :  15,000
구 :  종로구  / 법정동 :  충신동  / 도로명 :  율곡로23가길  / 면적 :  99.72  / 전세 :  11,000
구 :  종로구  / 법정동 :  충신동  / 도로명 :  율곡로23길  / 면적 :  99.72  / 전세 :  4,000
구 :  종로구  / 법정동 :  충신동  / 도로명 :  율곡로23길  / 면적 :  99.72  / 전세 :  17,500
구 :  종로구  / 법

 17%|█████████████▊                                                                     | 1/6 [02:06<10:34, 126.87s/it]

 종로구  / 법정동 :  충신동  / 도로명 :  율곡로23길  / 면적 :  99.72  / 전세 :  15,000
구 :  종로구  / 법정동 :  충신동  / 도로명 :  충신길  / 면적 :  99.72  / 전세 :  3,500
구 :  종로구  / 법정동 :  충신동  / 도로명 :  충신길  / 면적 :  99.72  / 전세 :  10,000
구 :  종로구  / 법정동 :  충신동  / 도로명 :  충신길  / 면적 :  99.72  / 전세 :  9,000
구 :  중랑구  / 법정동 :  면목동  / 도로명 :  겸재로  / 면적 :  99.72  / 전세 :  18,000
구 :  중랑구  / 법정동 :  면목동  / 도로명 :  겸재로  / 면적 :  99.72  / 전세 :  18,000
구 :  중랑구  / 법정동 :  면목동  / 도로명 :  겸재로  / 면적 :  99.72  / 전세 :  11,000
구 :  중랑구  / 법정동 :  면목동  / 도로명 :  겸재로  / 면적 :  99.72  / 전세 :  11,000
구 :  중랑구  / 법정동 :  면목동  / 도로명 :  겸재로  / 면적 :  99.72  / 전세 :  13,000
구 :  중랑구  / 법정동 :  면목동  / 도로명 :  겸재로  / 면적 :  99.72  / 전세 :  15,000
구 :  중랑구  / 법정동 :  면목동  / 도로명 :  겸재로  / 면적 :  99.72  / 전세 :  6,500
구 :  중랑구  / 법정동 :  면목동  / 도로명 :  겸재로10가길  / 면적 :  99.72  / 전세 :  15,000
구 :  중랑구  / 법정동 :  면목동  / 도로명 :  겸재로10길  / 면적 :  99.72  / 전세 :  12,000
구 :  중랑구  / 법정동 :  면목동  / 도로명 :  겸재로10길  / 면적 :  99.72  / 전세 :  6,500
구 :  중랑구  / 법정동 :  면목동  / 도로명 :  겸재로10길  / 

구 :  중랑구  / 법정동 :  면목동  / 도로명 :  동일로99길  / 면적 :  99.72  / 전세 :  3,000
구 :  중랑구  / 법정동 :  면목동  / 도로명 :  동일로99길  / 면적 :  99.72  / 전세 :  10,000
구 :  중랑구  / 법정동 :  면목동  / 도로명 :  동일로99길  / 면적 :  99.72  / 전세 :  3,000
구 :  중랑구  / 법정동 :  면목동  / 도로명 :  망우로30길  / 면적 :  99.72  / 전세 :  4,000
구 :  중랑구  / 법정동 :  면목동  / 도로명 :  망우로30길  / 면적 :  99.72  / 전세 :  9,000
구 :  중랑구  / 법정동 :  면목동  / 도로명 :  망우로30길  / 면적 :  99.72  / 전세 :  10,000
구 :  중랑구  / 법정동 :  면목동  / 도로명 :  망우로30길  / 면적 :  99.72  / 전세 :  4,500
구 :  중랑구  / 법정동 :  면목동  / 도로명 :  망우로30길  / 면적 :  99.72  / 전세 :  4,500
구 :  중랑구  / 법정동 :  면목동  / 도로명 :  면목로  / 면적 :  99.72  / 전세 :  6,000
구 :  중랑구  / 법정동 :  면목동  / 도로명 :  면목로  / 면적 :  99.72  / 전세 :  1,000
구 :  중랑구  / 법정동 :  면목동  / 도로명 :  면목로  / 면적 :  99.72  / 전세 :  23,500
구 :  중랑구  / 법정동 :  면목동  / 도로명 :  면목로  / 면적 :  99.72  / 전세 :  14,000
구 :  중랑구  / 법정동 :  면목동  / 도로명 :  면목로  / 면적 :  99.72  / 전세 :  20,000
구 :  중랑구  / 법정동 :  면목동  / 도로명 :  면목로  / 면적 :  99.72  / 전세 :  21,000
구 :  중랑구  / 법정동 :  면목동  / 도로명 : 

...

 83%|██████████████████████████████████████████████████████████████████████              | 5/6 [06:32<01:15, 75.38s/it]

구 :  구로구  / 법정동 :  구로동  / 도로명 :  새말로6길  / 면적 :  99.72  / 전세 :  13,000
구 :  구로구  / 법정동 :  구로동  / 도로명 :  새말로6길  / 면적 :  99.72  / 전세 :  9,000
구 :  구로구  / 법정동 :  구로동  / 도로명 :  새말로6길  / 면적 :  99.72  / 전세 :  16,000
구 :  구로구  / 법정동 :  구로동  / 도로명 :  새말로6길  / 면적 :  99.72  / 전세 :  9,000
구 :  서초구  / 법정동 :  반포동  / 도로명 :  강남대로83길  / 면적 :  99.72  / 전세 :  6,000
구 :  서초구  / 법정동 :  반포동  / 도로명 :  강남대로83길  / 면적 :  99.72  / 전세 :  13,000
구 :  서초구  / 법정동 :  반포동  / 도로명 :  강남대로83길  / 면적 :  99.72  / 전세 :  9,000
구 :  서초구  / 법정동 :  반포동  / 도로명 :  강남대로83길  / 면적 :  99.72  / 전세 :  18,000
구 :  서초구  / 법정동 :  반포동  / 도로명 :  강남대로83길  / 면적 :  99.72  / 전세 :  18,000
구 :  서초구  / 법정동 :  반포동  / 도로명 :  동광로  / 면적 :  99.72  / 전세 :  30,000
구 :  서초구  / 법정동 :  반포동  / 도로명 :  동광로39길  / 면적 :  99.72  / 전세 :  20,000
구 :  서초구  / 법정동 :  반포동  / 도로명 :  사평대로53길  / 면적 :  99.72  / 전세 :  12,000
구 :  서초구  / 법정동 :  반포동  / 도로명 :  사평대로53길  / 면적 :  99.72  / 전세 :  12,000
구 :  서초구  / 법정동 :  반포동  / 도로명 :  사평대로53길  / 면적 :  99.72  / 전세 :  12,000
구 :  서초구 

구 :  서초구  / 법정동 :  반포동  / 도로명 :  서래로2길  / 면적 :  99.72  / 전세 :  25,000
구 :  서초구  / 법정동 :  반포동  / 도로명 :  서래로5길  / 면적 :  99.72  / 전세 :  31,000
구 :  서초구  / 법정동 :  반포동  / 도로명 :  주흥13길  / 면적 :  99.72  / 전세 :  9,000
구 :  서초구  / 법정동 :  반포동  / 도로명 :  주흥15길  / 면적 :  99.72  / 전세 :  14,000
구 :  서초구  / 법정동 :  반포동  / 도로명 :  주흥17길  / 면적 :  99.72  / 전세 :  15,000
구 :  서초구  / 법정동 :  반포동  / 도로명 :  주흥17길  / 면적 :  99.72  / 전세 :  15,000
구 :  서초구  / 법정동 :  반포동  / 도로명 :  주흥3길  / 면적 :  99.72  / 전세 :  26,000
구 :  서초구  / 법정동 :  반포동  / 도로명 :  주흥6길  / 면적 :  99.72  / 전세 :  9,000
구 :  서초구  / 법정동 :  반포동  / 도로명 :  주흥7길  / 면적 :  99.72  / 전세 :  11,000
구 :  서초구  / 법정동 :  반포동  / 도로명 :  주흥7길  / 면적 :  99.72  / 전세 :  8,000
구 :  서초구  / 법정동 :  반포동  / 도로명 :  주흥9길  / 면적 :  99.72  / 전세 :  7,500
구 :  서초구  / 법정동 :  반포동  / 도로명 :  주흥9길  / 면적 :  99.72  / 전세 :  11,000
구 :  서초구  / 법정동 :  반포동  / 도로명 :  주흥9길  / 면적 :  99.72  / 전세 :  16,000
구 :  서초구  / 법정동 :  반포동  / 도로명 :  주흥길  / 면적 :  99.72  / 전세 :  13,000
구 :  서초구  / 법정동 :  반포동  / 도로명 :  

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [07:11<00:00, 71.85s/it]

구 :  서초구  / 법정동 :  반포동  / 도로명 :  주흥길  / 면적 :  99.72  / 전세 :  10,000


In [17]:
df_houseRent_2019

,구,법정동,도로명,면적,전세
0,종로구,효제동,종로35길,99.72,"18,000"
1,종로구,효제동,대학로2길,99.72,"15,000"
2,종로구,효제동,대학로2길,99.72,"12,000"
3,종로구,효제동,대학로2길,99.72,"12,000"
4,종로구,종로6가,충신길,99.72,"13,000"
...,...,...,...,...,...
3288,서초구,반포동,주흥길,99.72,"20,000"
3289,서초구,반포동,주흥길,99.72,"21,000"
3290,서초구,반포동,주흥길,99.72,"30,000"
3291,서초구,반포동,주흥길,99.72,"45,000"


In [20]:
df_houseRent_2019[df_houseRent_2019["면적"].str.contains(",")]

,구,법정동,도로명,면적,전세


In [21]:
driver.close()

In [22]:
df_houseRent_2019.to_csv("data/분석용_데이터/단독_다가구_전세_결측치(cp949).csv", encoding = "cp949", index = False)

# 단독/다가구 월세

In [302]:
def house_monthly_info(gu, dong, result):
    df = pd.DataFrame(columns=["구", "법정동", "도로명", "면적", "보증금", "월세"])
    
    for i in range(len(result)):
        result_tmp = result[i].text.split()

        if result_tmp[0] != "단독다가구":
            upper_name = result_tmp[0]

        else:
            result_tmp.insert(0, upper_name)

        house_area = result_tmp[2]
        info_price = result_tmp[3:-1]
        info_length = len(info_price) // 3

        for k in range(info_length):
            deposit = info_price[info_length]
            monthly = info_price[info_length+1]
            
            df= df.append({"구" : gu, "법정동" : dong, "도로명" : upper_name,
                           "면적" : house_area, "보증금" : deposit, "월세" : monthly}, ignore_index=True)
    return df

In [303]:
driver = wd.Chrome(executable_path="chrome/chromedriver.exe")
driver.implicitly_wait(3)

apt_url = "http://land.seoul.go.kr/land/rtms/rtmsSingleHouse.do" 
driver.get(apt_url)
page = urlopen(apt_url)

xp = """//*[@id="content"]/div[3]/div[1]/section/ul/li[2]/a"""
driver.find_element_by_xpath(xp).click()
time.sleep(2)

kinds = "월세"
driver.find_element_by_id("selectJWGubun").send_keys(kinds)

In [197]:
# 데이터 프레임 준비
df_houseMonthly_2019 = pd.DataFrame(columns = ["구", "법정동", "도로명", "면적", "보증금", "월세"])

In [198]:
for gu in tqdm(singleHouse_rent.keys()):
    driver.find_element_by_id("selectJRGubun").send_keys("지번")
    time.sleep(2)

    # 지역선택 - 구 선택
    driver.find_element_by_id("selectSigungu").send_keys(gu)
    time.sleep(2)
    dong_list = singleHouse_rent[gu]
    
    for dong in dong_list:
        # 지역선택 - 동 선택
        driver.find_element_by_id("selectBjdong").send_keys(dong)
        time.sleep(2)

        # 기간선택 - 2019년
        driver.find_element_by_id("selectGubun").send_keys("분기")
        time.sleep(2)
        driver.find_element_by_id("selectYear").send_keys("2019")
        time.sleep(2)
        
        for i in range(1, 5):
            driver.find_element_by_id("selectBoongi").send_keys(str(i)+"분기")
            time.sleep(2)
            # 조회버튼 - click
            xp = """//*[@id="search"]"""
            driver.find_element_by_xpath(xp).click()
            time.sleep(2)
            
            # 데이터 - table 전부 가져오기
            result_raw = driver.find_element_by_id("resultList")

            # 데이터 - row별로 가져오기
            result = result_raw.find_elements_by_tag_name("tr")
            
            if result[0].text == '조회된 내용이 없습니다.':
                continue
            
            # 데이터 - 가공한 후 데이터프레임에 넣기
            new_df = house_monthly_info(gu, dong, result)
            
            # 수집된 데이터를 df_aptSales_2019 뒤로 계속 붙여넣기
            df_houseMonthly_2019 = pd.concat([df_houseMonthly_2019, new_df], ignore_index=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [04:17<00:00, 42.91s/it]


In [307]:
df_houseMonthly_2019[df_houseMonthly_2019["월세"] == "08.31"]

,구,법정동,도로명,면적,보증금,월세
401,구로구,구로동,구로동로21길,20,40,08.31
402,구로구,구로동,구로동로21길,20,40,08.31


In [317]:
df_houseMonthly_2019.loc[401:402, "보증금"] = 300
df_houseMonthly_2019.loc[401, "월세"] = 40
df_houseMonthly_2019.loc[402, "월세"] = 32

In [319]:
df_houseMonthly_2019.to_csv("data/분석용_데이터/단독_다가구_월세_결측치(cp949).csv", encoding = "cp949", index = False)

# 컬럼 정리하기

#### 매매

In [33]:
df_houseSales_2019 = pd.read_csv("data/분석용_데이터/단독_다가구_매매_결측치(cp949).csv", encoding = "cp949")
df_houseSales_2019.head()

,구,법정동,도로명,면적,가격
0,종로구,연지동,창경궁로16나길,24.79,"22,460"
1,종로구,효제동,대학로2길,24.76,"42,000"
2,종로구,효제동,대학로2길,120.35,"125,000"
3,종로구,효제동,율곡로14길,305.00,"139,531"
4,종로구,효제동,김상옥로,49.59,"178,500"


In [34]:
# 가격 , 삭제하고 int형으로!
df_houseSales_2019["가격"] = df_houseSales_2019["가격"].str.replace(",", "")
df_houseSales_2019["가격"] = df_houseSales_2019["가격"].astype("int")

# 단위면적당 금액 column생성
df_houseSales_2019["단위면적당금액"] = df_houseSales_2019["가격"] / df_houseSales_2019["면적"]

df_houseSales_2019.head()

,구,법정동,도로명,면적,가격,단위면적당금액
0,종로구,연지동,창경궁로16나길,24.79,22460,906.010488
1,종로구,효제동,대학로2길,24.76,42000,1696.284330
2,종로구,효제동,대학로2길,120.35,125000,1038.637308
3,종로구,효제동,율곡로14길,305.00,139531,457.478689
4,종로구,효제동,김상옥로,49.59,178500,3599.516031


In [35]:
# 카카오 API에 들어갈 수 있는 column 만들기
df_houseSales_2019["도로명주소"] = "서울특별시" + " " + df_houseSales_2019["구"] + " " + df_houseSales_2019["법정동"] + " " + df_houseSales_2019["도로명"]

df_houseSales_2019.head()

,구,법정동,도로명,면적,가격,단위면적당금액,도로명주소
0,종로구,연지동,창경궁로16나길,24.79,22460,906.010488,서울특별시 종로구 연지동 창경궁로16나길
1,종로구,효제동,대학로2길,24.76,42000,1696.284330,서울특별시 종로구 효제동 대학로2길
2,종로구,효제동,대학로2길,120.35,125000,1038.637308,서울특별시 종로구 효제동 대학로2길
3,종로구,효제동,율곡로14길,305.00,139531,457.478689,서울특별시 종로구 효제동 율곡로14길
4,종로구,효제동,김상옥로,49.59,178500,3599.516031,서울특별시 종로구 효제동 김상옥로


In [59]:
# 컬럼 정리
df_houseSales_2019 = df_houseSales_2019.drop(["면적", "가격"], axis = 1)
df_houseSales_2019.head()

,구,법정동,도로명,단위면적당금액,도로명주소
0,종로구,연지동,창경궁로16나길,906.010488,서울특별시 종로구 연지동 창경궁로16나길
1,종로구,효제동,대학로2길,1696.284330,서울특별시 종로구 효제동 대학로2길
2,종로구,효제동,대학로2길,1038.637308,서울특별시 종로구 효제동 대학로2길
3,종로구,효제동,율곡로14길,457.478689,서울특별시 종로구 효제동 율곡로14길
4,종로구,효제동,김상옥로,3599.516031,서울특별시 종로구 효제동 김상옥로


In [61]:
df_houseSales_2019

,구,법정동,도로명,단위면적당금액,도로명주소
0,종로구,연지동,창경궁로16나길,906.010488,서울특별시 종로구 연지동 창경궁로16나길
1,종로구,효제동,대학로2길,1696.284330,서울특별시 종로구 효제동 대학로2길
2,종로구,효제동,대학로2길,1038.637308,서울특별시 종로구 효제동 대학로2길
3,종로구,효제동,율곡로14길,457.478689,서울특별시 종로구 효제동 율곡로14길
4,종로구,효제동,김상옥로,3599.516031,서울특별시 종로구 효제동 김상옥로
...,...,...,...,...,...
1493,송파구,가락동,송이로20길,413.384471,서울특별시 송파구 가락동 송이로20길
1494,송파구,가락동,중대로10길,424.112657,서울특별시 송파구 가락동 중대로10길
1495,송파구,가락동,중대로10길,402.396653,서울특별시 송파구 가락동 중대로10길
1496,송파구,가락동,송이로23길,338.848211,서울특별시 송파구 가락동 송이로23길


In [60]:
# 저장
df_houseSales_2019.to_csv("data/분석용_데이터/단독_다가구_매매_결측치_forAPI(cp949).csv", encoding = "cp949", index=False)

#### 전세

In [41]:
df_houseRent_2019 = pd.read_csv("data/분석용_데이터/단독_다가구_전세_결측치(cp949).csv", encoding = "cp949")
df_houseRent_2019.head()

,구,법정동,도로명,면적,전세
0,종로구,효제동,종로35길,99.72,"18,000"
1,종로구,효제동,대학로2길,99.72,"15,000"
2,종로구,효제동,대학로2길,99.72,"12,000"
3,종로구,효제동,대학로2길,99.72,"12,000"
4,종로구,종로6가,충신길,99.72,"13,000"


In [42]:
# 전세 , 삭제하고 int형으로!
df_houseRent_2019["전세"] = df_houseRent_2019["전세"].str.replace(",", "")
df_houseRent_2019["전세"] = df_houseRent_2019["전세"].astype("int")

# 단위면적당 금액 column생성
df_houseRent_2019["단위면적당_전세"] = df_houseRent_2019["전세"] / df_houseRent_2019["면적"]

df_houseRent_2019.head()

,구,법정동,도로명,면적,전세,단위면적당_전세
0,종로구,효제동,종로35길,99.72,18000,180.505415
1,종로구,효제동,대학로2길,99.72,15000,150.421179
2,종로구,효제동,대학로2길,99.72,12000,120.336943
3,종로구,효제동,대학로2길,99.72,12000,120.336943
4,종로구,종로6가,충신길,99.72,13000,130.365022


In [43]:
# 카카오 API에 들어갈 수 있는 column 만들기
df_houseRent_2019["도로명주소"] = "서울특별시" + " " + df_houseRent_2019["구"] + " " + df_houseRent_2019["법정동"] + " " + df_houseRent_2019["도로명"]

df_houseRent_2019.head()

,구,법정동,도로명,면적,전세,단위면적당_전세,도로명주소
0,종로구,효제동,종로35길,99.72,18000,180.505415,서울특별시 종로구 효제동 종로35길
1,종로구,효제동,대학로2길,99.72,15000,150.421179,서울특별시 종로구 효제동 대학로2길
2,종로구,효제동,대학로2길,99.72,12000,120.336943,서울특별시 종로구 효제동 대학로2길
3,종로구,효제동,대학로2길,99.72,12000,120.336943,서울특별시 종로구 효제동 대학로2길
4,종로구,종로6가,충신길,99.72,13000,130.365022,서울특별시 종로구 종로6가 충신길


In [54]:
# 컬럼 정리
df_houseRent_2019 = df_houseRent_2019.drop(["면적", "전세"], axis = 1)
df_houseRent_2019.head()

,구,법정동,도로명,단위면적당_전세,도로명주소
0,종로구,효제동,종로35길,180.505415,서울특별시 종로구 효제동 종로35길
1,종로구,효제동,대학로2길,150.421179,서울특별시 종로구 효제동 대학로2길
2,종로구,효제동,대학로2길,120.336943,서울특별시 종로구 효제동 대학로2길
3,종로구,효제동,대학로2길,120.336943,서울특별시 종로구 효제동 대학로2길
4,종로구,종로6가,충신길,130.365022,서울특별시 종로구 종로6가 충신길


#### 월세

In [26]:
df_houseMonthly_2019 = pd.read_csv("data/분석용_데이터/단독_다가구_월세_결측치(cp949).csv", encoding = "cp949")
df_houseMonthly_2019.head()

,구,법정동,도로명,면적,보증금,월세
0,종로구,충신동,충신4길,14.88,100,20.0
1,종로구,충신동,충신4길,16.52,200,20.0
2,종로구,충신동,충신4길,16.66,250,28.0
3,종로구,충신동,충신4나길,28.03,500,45.0
4,중랑구,면목동,겸재로,10.00,120,20.0


In [47]:
# 보증금 , 삭제하고 int형으로!
df_houseMonthly_2019["보증금"] = df_houseMonthly_2019["보증금"].str.replace(",", "")
df_houseMonthly_2019["보증금"] = df_houseMonthly_2019["보증금"].astype("int")

df_houseMonthly_2019.head()

,구,법정동,도로명,면적,보증금,월세
0,종로구,충신동,충신4길,14.88,100,20.0
1,종로구,충신동,충신4길,16.52,200,20.0
2,종로구,충신동,충신4길,16.66,250,28.0
3,종로구,충신동,충신4나길,28.03,500,45.0
4,중랑구,면목동,겸재로,10.00,120,20.0


In [48]:
### 전환율 : {연립다세대 : 4.48, 아파트 : 4.04, 단독주택 : 6.13}- 데이터에 따라 변환 할 것
df_houseMonthly_2019["전세보증금"] = ((df_houseMonthly_2019["월세"] * 12 / 6.13) * 100) + df_houseMonthly_2019['보증금']
df_houseMonthly_2019.head()

,구,법정동,도로명,면적,보증금,월세,전세보증금
0,종로구,충신동,충신4길,14.88,100,20.0,4015.171289
1,종로구,충신동,충신4길,16.52,200,20.0,4115.171289
2,종로구,충신동,충신4길,16.66,250,28.0,5731.239804
3,종로구,충신동,충신4나길,28.03,500,45.0,9309.135400
4,중랑구,면목동,겸재로,10.00,120,20.0,4035.171289


In [49]:
# 단위면적당 금액 column생성
df_houseMonthly_2019["단위면적당_전세"] = df_houseMonthly_2019["전세보증금"] / df_houseMonthly_2019["면적"]

df_houseMonthly_2019.head()

,구,법정동,도로명,면적,보증금,월세,전세보증금,단위면적당_전세
0,종로구,충신동,충신4길,14.88,100,20.0,4015.171289,269.836780
1,종로구,충신동,충신4길,16.52,200,20.0,4115.171289,249.102378
2,종로구,충신동,충신4길,16.66,250,28.0,5731.239804,344.011993
3,종로구,충신동,충신4나길,28.03,500,45.0,9309.135400,332.113286
4,중랑구,면목동,겸재로,10.00,120,20.0,4035.171289,403.517129


In [52]:
# 컬럼 정리
df_houseMonthly_2019 = df_houseMonthly_2019.drop(["면적", "보증금", "월세", "전세보증금"], axis = 1)
df_houseMonthly_2019.head()

,구,법정동,도로명,단위면적당_전세
0,종로구,충신동,충신4길,269.836780
1,종로구,충신동,충신4길,249.102378
2,종로구,충신동,충신4길,344.011993
3,종로구,충신동,충신4나길,332.113286
4,중랑구,면목동,겸재로,403.517129


In [53]:
# 카카오 API에 들어갈 수 있는 column 만들기
df_houseMonthly_2019["도로명주소"] = "서울특별시" + " " + df_houseMonthly_2019["구"] + " " + df_houseMonthly_2019["법정동"] + " " + df_houseMonthly_2019["도로명"]

df_houseMonthly_2019.head()

,구,법정동,도로명,단위면적당_전세,도로명주소
0,종로구,충신동,충신4길,269.836780,서울특별시 종로구 충신동 충신4길
1,종로구,충신동,충신4길,249.102378,서울특별시 종로구 충신동 충신4길
2,종로구,충신동,충신4길,344.011993,서울특별시 종로구 충신동 충신4길
3,종로구,충신동,충신4나길,332.113286,서울특별시 종로구 충신동 충신4나길
4,중랑구,면목동,겸재로,403.517129,서울특별시 중랑구 면목동 겸재로


In [57]:
df_houseRent_2019.shape, df_houseMonthly_2019.shape

((3293, 5), (557, 5))

#### 전세 월세 합쳐서 저장

In [58]:
df_houseRentAll_2019 = pd.concat([df_houseRent_2019, df_houseMonthly_2019])
df_houseRentAll_2019.to_csv("data/분석용_데이터/단독_다가구_전월세_결측치_forAPI(cp949).csv", encoding = "cp949", index=False)